# Using NEAT for XOR Solver Optimization

This notebook explores the application of Neuroevolution of Augmenting Topologies (NEAT) as an XOR solver.

In [1]:
# impor the standard library and the NEAT python library
import os
import shutil
import neat
import visualize

In [2]:
# the current working directory
# local_dir = os.path.dirname(__file__)
local_dir = os.getcwd()
# the directory to store outputs
out_dir = os.path.join(local_dir, 'out')

In [3]:
# The XOR inputs and expected corresponding outputs for fitness evaluation
xor_inputs = [(0.0, 0.0),
              (0.0, 1.0),
              (1.0, 0.0),
              (1.0, 1.0)]
xor_outputs = [(0.0),
               (1.0),
               (1.0),
               (0.0)]

In [4]:
def eval_fitness(net) -> int:
    """
    Evaluate fitness of the genome that was used to 
    generate provided net

    Arguments:
        net:
            The feed-forward neural network generated from genome
    
    Returns:
        fitness:int
            The fitness score - the higher score the means 
            the better fit organism. Maximal score: 16.0
    """
    # initialize error_sum
    error_sum = 0.0

    # loop through the list of xor inputs and outputs and check the model's performance
    for xi, xo in zip(xor_inputs, xor_outputs):
        output = net.activate(xi)
        # print(f'output is {output[0]}')
        # print(f'xor_output is {xo[0]}')
        error_sum += abs(output[0]-xo)

    # functional implementation of the above code
    # outputs = map(net.activate, xor_inputs)
    # errors = map(lambda x, y: abs(x-y), xor_outputs, outputs)
    # error_sum = sum(errors)


    # calculate amplified fitness
    fitness = (4 - error_sum)**2
    # print('works here')
    return fitness

In [5]:
def eval_genomes(genomes, config) -> None:
    """
    The function to evaluate the fitnes of each genome in the genome list.

    The provided configuration is used to create feed-forward neural 
    network from each genome and after that created the neural network 
    evaluated in its ability to solve XOR problem. As a result of this 
    function execution, the fitness score of each genome updated to the
    newly evaluated one.

    Arguments:
        genomes:
            the list of genomes from population in the current generation

        config:
            the configuration settings with algorithm hyper-parameters

    Returns:
        None
    """

    for genome_id, genome in genomes:
        genome.fitness = 4.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        genome.fitness = eval_fitness(net)

In [6]:
# def run_experiment(config_file) -> None:
#     """
#     The function to run XOR experiment against hyper-parameters
#     defined in the provided configuration file.

#     The winner genome will be rendered as a graph as well as the
#     important statistics of neuroevolition process execution

#     Arguments:
#         config_file:str
#             the path to the file with experiment configurations

#     Returns:
#         None

#     """

#     # load configuration
#     config = neat.Config(neat.DefaultGenome,
#                         neat.DefaultReproduction,
#                         neat.DefaultSpeciesSet,
#                         neat.DefaultStagnation,
#                         config_file)
    
#     # create the population, which is the top-level objects for a NEAT run
#     p = neat.Population(config)

#     # add a stdout reporter to show progres
#     p.add_reporter(neat.StdOutReporter(True))
#     stats = neat.StatisticsReporter()
#     p.add_reporter(stats)
#     p.add_reporter(neat.Checkpointer(5, filename_prefix='out/neat-checkpoint-'))

#     # run for up to 300 generations
#     best_genome = (p.run, eval_genomes, 300)

#     # display the best genome among generations
#     print(f'\nBest genome:\n{best_genome}')

#     # show output of the most fit genome against training data
#     print('\nOutput:')
#     net = neat.nn.FeedForwardNetwork.create(best_genome, config)
#     for xi, xo in zip(xor_inputs, xor_outputs):
#         output = net.activate(xi)
#         print('input {!r}, expected output {!r}, got {!r}'.format(xi, xo, output))
    
#     # below is a functional implementation of the above code
#     # outputs = map(net.activate, xor_inputs)
#     # map(lambda x, y, z: print(f'input {x}, expected output {y}, got {z}'), xor_inputs, xor_outputs, outputs)


#     # check if the best genome is an adequate XOR solver
#     best_genome_fitness = eval_fitness(net)
#     if best_genome_fitness > config.fitness_threshold:
#         print('\n\nSUCESS: The XOR problem solve found.')
#     else:
#         print('\n\nFAILURE: Failed to find XOR problem solver.')

#     # visualize the experiment results
#     node_names = {-1:'A', -2:'B', 0:'A XOR B'}
#     visualize.draw_net(config, 
#                        best_genome, 
#                        True, 
#                        node_names = node_names, 
#                        directory = out_dir)
#     visualize.plot_stats(stats,
#                          ylog = False,
#                          filaname = os.path.join(out_dir, 'avg_fitness.svg'))
#     visualize.plot_species(stats,
#                            view = True,
#                            filename = os.path.join(out_dir, 'speciation.svg'))

In [7]:
def run_experiment(config_file):
    """
    The function to run XOR experiment against hyper-parameters 
    defined in the provided configuration file.
    The winner genome will be rendered as a graph as well as the
    important statistics of neuroevolution process execution.
    Arguments:
        config_file: the path to the file with experiment 
                    configuration
    """
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix='out/neat-checkpoint-'))

    # Run for up to 300 generations.
    best_genome = p.run(eval_genomes, 300)

    # Display the best genome among generations.
    print('\nBest genome:\n{!s}'.format(best_genome))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    net = neat.nn.FeedForwardNetwork.create(best_genome, config)
    for xi, xo in zip(xor_inputs, xor_outputs):
        output = net.activate(xi)
        print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

    # Check if the best genome is an adequate XOR solver
    best_genome_fitness = eval_fitness(net)
    print('works here')
    print(best_genome_fitness)
    if best_genome_fitness > config.fitness_threshold:
        print("\n\nSUCCESS: The XOR problem solver found!!!")
    else:
        print("\n\nFAILURE: Failed to find XOR problem solver!!!")

    # Visualize the experiment results
    # node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
    # visualize.draw_net(config, best_genome, True, node_names=node_names, directory=out_dir)
    # visualize.plot_stats(stats, ylog=False, view=True, filename=os.path.join(out_dir, 'avg_fitness.svg'))
    # visualize.plot_species(stats, view=True, filename=os.path.join(out_dir, 'speciation.svg'))

    print('works here. Great!')

In [8]:
def clean_output() -> None:
    """
    This function cleans the results of any previous run (if any) or 
    init the ouput directory

    Arguments:
        None

    Returns:
        None
    """

    if os.path.isdir(out_dir):
        # remove files from previous run
        shutil.rmtree(out_dir)
    
    # create the output directory
    os.makedirs(out_dir, exist_ok = False)

In [9]:
if __name__ == '__main__':
    # determine the path to configuration file. This path manipulation is
    # is here so that the script will run successfully regardless of
    # the current working directory
    config_path = os.path.join(local_dir, 'xor_config.ini')

    # clean results of previous run if any or init the output directory
    clean_output()

    # run the experiment
    run_experiment(config_path)


 ****** Running generation 0 ****** 

Population's average fitness: 3.94925 stdev: 1.12406
Best fitness: 8.40700 - size: (1, 2) - species 1 - id 126
Average adjusted fitness: 0.358
Mean genetic distance 1.163, standard deviation 0.423
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   150      8.4    0.358     0
Total extinctions: 0
Generation time: 0.039 sec

 ****** Running generation 1 ****** 

Population's average fitness: 4.29449 stdev: 1.09522
Best fitness: 8.40700 - size: (1, 2) - species 1 - id 126
Average adjusted fitness: 0.431
Mean genetic distance 1.254, standard deviation 0.458
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   150      8.4    0.431     1
Total extinctions: 0
Generation time: 0.047 sec (0.043 average)

 ****** Running generation 2 ****** 

Population's average fitness: 4.44846 stdev: 

Population's average fitness: 5.18737 stdev: 2.15663
Best fitness: 8.99393 - size: (2, 4) - species 1 - id 1220
Average adjusted fitness: 0.505
Mean genetic distance 1.882, standard deviation 0.659
Population of 150 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    9    85      9.0    0.524     1
     2    1    65      9.0    0.486     0
Total extinctions: 0
Generation time: 0.056 sec (0.047 average)
Saving checkpoint to out/neat-checkpoint-9

 ****** Running generation 10 ****** 

Population's average fitness: 5.41011 stdev: 2.21300
Best fitness: 8.99710 - size: (2, 4) - species 2 - id 1495
Average adjusted fitness: 0.553
Mean genetic distance 1.735, standard deviation 0.685
Population of 150 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   10    91      9.0    0.538     0
     2    2    59      9.0    0.569     0
Total extinctions: 0
Generation time: 0.0

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [ ]:
test1 = [1,2,3]
test2 = ['a', 'b', 'c']
test3 = [10, 9, 8]
test4 = map(lambda x, y, z: print(f'input {x}, expected output {y}, got {z}'), test1, test2, test3)

In [ ]:
print(list(test4))

input 1, expected output a, got 10
input 2, expected output b, got 9
input 3, expected output c, got 8
[None, None, None]


In [ ]:
list(test4)

input 1, expected output a, got 10
input 2, expected output b, got 9
input 3, expected output c, got 8


[None, None, None]

In [ ]:
test5 = list(test4)

input 1, expected output a, got 10
input 2, expected output b, got 9
input 3, expected output c, got 8
